In [ ]:
from google.colab import drive
#import os, sys
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
!pip install mxnet-cu101mkl
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

     |████████████████████████████████| 712.3MB 25kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 18.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595679 sha256=cba9e0fca14775fcb4c1cafe4dcd146654659cbade52cc12a0cdedaf83f67db7
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 20.4MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zpwuwkph
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zpwuwkph
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=c31a939bc2930064c4b4f4de5e956ce33c5d6b3dd8c2ad6759a0987db0278cf9
  Stored in directory: /tmp/pip-ephem-wheel-cache-dxxwbym8/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import pandas as pd
import numpy as np
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random

from kobert.mxnet_kobert import get_mxnet_kobert_model
from kobert.utils import get_tokenizer

### Loading KoBERT

In [ ]:
ctx = mx.gpu()

In [ ]:
bert_base, vocab = get_mxnet_kobert_model(use_decoder=False, use_classifier=False, ctx=ctx)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
ds = gluon.data.SimpleDataset([['나 보기가 역겨워', '김소월']])
trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=10)

list(ds.transform(trans))

[(array([   2, 1370, 2362, 5330, 3322,    3, 1316, 6607, 7028,    3],
        dtype=int32),
  array(10, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int32))]

### Loading Data


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/problem_complete.csv')

###train data, test data 분류

In [ ]:
train_df, remaining = train_test_split(df, random_state = 52, train_size = 0.7, stratify = df.tag.values)
valid_df, _ = train_test_split(remaining, random_state=52, train_size = 0.2, stratify = remaining.tag.values)

In [ ]:
print(train_df.shape, valid_df.shape)

#라벨의 숫자화
label = df['tag'].values
encoder = LabelEncoder()
encoder.fit(label)

(3952, 7) (338, 7)


LabelEncoder()

###input data로 변환 (문제, label)

In [ ]:
dataset_train = []
dataset_test = []

tmp = encoder.transform(train_df['tag'])
i = 0
for index, row in train_df.iterrows():
  problem = row['description']
  if not pd.isna(row['input']) :
    problem = problem + row['input'] 
  if not pd.isna(row['output']) :
      problem = problem + row['output'] 
  dataset_train.append([problem , tmp[i]])
  i = i + 1

i = 0
tmp = encoder.transform(valid_df['tag'])
for index, row in valid_df.iterrows():
  problem = row['description']
  if not pd.isna(row['input']) :
    problem = problem + row['input'] 
  if not pd.isna(row['output']) :
      problem = problem + row['output'] 
  dataset_test.append([problem , tmp[i]])
  i = i + 1



###Bert에 맞게 input 변환

In [ ]:
class BERTDataset(mx.gluon.data.Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        sent_dataset = gluon.data.SimpleDataset([[
            i[sent_idx],
        ] for i in dataset])
        self.sentences = sent_dataset.transform(transform)
        self.labels = gluon.data.SimpleDataset(
            [np.array(np.int32(i[label_idx])) for i in dataset])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
max_len = 512 

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

###Bert 다음 layer 설정 (single + dropout 추가)

In [ ]:
class BERTClassifier(nn.Block):
    def __init__(self,
                 bert,
                 num_classes=2,#change
                 dropout=None,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            self.classifier.add(nn.Dense(256)) # add layer
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):
        _, pooler = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler)
                                           

In [ ]:
model = BERTClassifier(bert_base, num_classes=11, dropout=0.1)
# 분류 레이어만 초기화 한다. 
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize()

# softmax cross entropy loss for classification
loss_function = gluon.loss.SoftmaxCELoss()

metric = mx.metric.Accuracy()

In [ ]:
batch_size = 8
lr = 5e-5

train_dataloader = mx.gluon.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=int(batch_size/2), num_workers=5)

In [ ]:
trainer = gluon.Trainer(model.collect_params(), 'bertadam',
                        {'learning_rate': lr, 'epsilon': 1e-9, 'wd':0.01})

log_interval = 4
num_epochs = 5

In [ ]:
# LayerNorm과 Bias에는 Weight Decay를 적용하지 않는다. 
for _, v in model.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
params = [
    p for p in model.collect_params().values() if p.grad_req != 'null'
]


In [ ]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    i = 0
    for i, (t,v,s, label) in enumerate(data_iter):
        token_ids = t.as_in_context(ctx)
        valid_length = v.as_in_context(ctx)
        segment_ids = s.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = model(token_ids, segment_ids, valid_length.astype('float32'))
        acc.update(preds=output, labels=label)
        if i > 1000:
            break
        i += 1
    return(acc.get()[1])

In [ ]:
#learning rate warmup을 위한 준비 
accumulate = 4
step_size = batch_size * accumulate if accumulate else batch_size
num_train_examples = len(data_train)
num_train_steps = int(num_train_examples / step_size * num_epochs)
warmup_ratio = 0.1
num_warmup_steps = int(num_train_steps * warmup_ratio)
step_num = 0
all_model_params = model.collect_params()

In [ ]:
# Set grad_req if gradient accumulation is required
if accumulate and accumulate > 1:
    for p in params:
        p.grad_req = 'add'

In [ ]:
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps - num_warmup_steps)
            new_lr = lr - offset * lr
        trainer.set_learning_rate(new_lr)
        with mx.autograd.record():
            # load data to GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # backward computation
        ls.backward()
        if not accumulate or (batch_id + 1) % accumulate == 0:
          trainer.allreduce_grads()
          nlp.utils.clip_grad_global_norm(params, 1)
          trainer.update(accumulate if accumulate else 1)
          step_num += 1
          if accumulate and accumulate > 1:
              # set grad to zero for gradient accumulation
              all_model_params.zero_grad()

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (50) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.10f}, acc={:.3f}'
                         .format(epoch_id + 1, batch_id + 1, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    test_acc = evaluate_accuracy(model, test_dataloader, ctx)
    print('Test Acc : {}'.format(test_acc))

[Epoch 1 Batch 50/494] loss=29.5242, lr=0.0000098361, acc=0.105
[Epoch 1 Batch 100/494] loss=27.0090, lr=0.0000196721, acc=0.171
[Epoch 1 Batch 150/494] loss=26.5874, lr=0.0000303279, acc=0.193
[Epoch 1 Batch 200/494] loss=25.8800, lr=0.0000401639, acc=0.206
[Epoch 1 Batch 250/494] loss=26.3430, lr=0.0000499101, acc=0.211
[Epoch 1 Batch 300/494] loss=27.1371, lr=0.0000488309, acc=0.214
[Epoch 1 Batch 350/494] loss=27.0891, lr=0.0000476619, acc=0.216
[Epoch 1 Batch 400/494] loss=26.8233, lr=0.0000465827, acc=0.221
[Epoch 1 Batch 450/494] loss=26.3353, lr=0.0000454137, acc=0.221
Test Acc : 0.026627218934911243
[Epoch 2 Batch 50/494] loss=25.7168, lr=0.0000433453, acc=0.275
[Epoch 2 Batch 100/494] loss=26.0570, lr=0.0000422662, acc=0.256
[Epoch 2 Batch 150/494] loss=26.6398, lr=0.0000410971, acc=0.247
[Epoch 2 Batch 200/494] loss=25.8749, lr=0.0000400180, acc=0.242
[Epoch 2 Batch 250/494] loss=26.2266, lr=0.0000388489, acc=0.241
[Epoch 2 Batch 300/494] loss=27.0330, lr=0.0000377698, acc=0